In [ ]:
import httpx
import polars as pl
from google.cloud import storage
import os
from datetime import datetime

# Configuration
NOETL_API = "http://localhost:8082"
PLAYBOOK_PATH = "tests/postgres_excel_gcs"
GCS_BUCKET = "noetl-test-exports"
EXCEL_FILENAME = "test_export.xlsx"

## 1. Find Latest Execution

In [ ]:
# Query for latest execution of this playbook
query = f"""
SELECT execution_id, catalog_id, created_at, status
FROM noetl.event
WHERE event_type = 'playbook.initialized'
  AND node_name = '{PLAYBOOK_PATH}'
ORDER BY execution_id DESC
LIMIT 1
"""

response = httpx.post(
    f"{NOETL_API}/api/postgres/execute",
    json={"query": query, "schema": "noetl"}
)

result = response.json()
if result['result']:
    execution_id = result['result'][0][0]
    status = result['result'][0][3]
    print(f"Latest execution: {execution_id}")
    print(f"Status: {status}")
else:
    print("No execution found. Run the test first!")
    execution_id = None

## 2. Check Execution Events

In [ ]:
if execution_id:
    query = f"""
    SELECT event_type, node_name, status, duration, error
    FROM noetl.event
    WHERE execution_id = {execution_id}
    ORDER BY event_id
    """
    
    response = httpx.post(
        f"{NOETL_API}/api/postgres/execute",
        json={"query": query, "schema": "noetl"}
    )
    
    events = response.json()['result']
    
    print("\nExecution Timeline:")
    print("-" * 80)
    for event in events:
        event_type, node_name, status, duration, error = event
        duration_str = f"{duration}ms" if duration else "-"
        error_str = f" ERROR: {error}" if error else ""
        print(f"{event_type:25s} {node_name:30s} {status:12s} {duration_str:10s}{error_str}")
    
    # Check for failures
    failed_events = [e for e in events if e[4]]  # Check error field
    if failed_events:
        print("\n⚠️  FAILURES DETECTED:")
        for event in failed_events:
            print(f"  - {event[1]}: {event[4]}")
    else:
        print("\n✓ No errors detected")

## 3. Download Excel File from GCS

In [ ]:
# Initialize GCS client
storage_client = storage.Client()
bucket = storage_client.bucket(GCS_BUCKET)

# Download file
gcs_path = f"exports/{EXCEL_FILENAME}"
local_path = f"/tmp/{EXCEL_FILENAME}"

try:
    blob = bucket.blob(gcs_path)
    blob.download_to_filename(local_path)
    
    file_size = os.path.getsize(local_path)
    print(f"✓ Downloaded: gs://{GCS_BUCKET}/{gcs_path}")
    print(f"  File size: {file_size:,} bytes")
    print(f"  Local path: {local_path}")
except Exception as e:
    print(f"✗ Failed to download: {e}")
    local_path = None

## 4. Inspect Excel Sheets

In [ ]:
if local_path and os.path.exists(local_path):
    import openpyxl
    
    workbook = openpyxl.load_workbook(local_path)
    
    print("\nExcel Structure:")
    print("-" * 80)
    print(f"Total sheets: {len(workbook.sheetnames)}")
    print(f"Sheet names: {', '.join(workbook.sheetnames)}")
    
    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
        rows = sheet.max_row
        cols = sheet.max_column
        print(f"\n  {sheet_name}: {rows} rows × {cols} columns")

## 5. Verify Sheet Contents with Polars

In [ ]:
if local_path and os.path.exists(local_path):
    # Read all sheets
    sheets = {
        'Employees': pl.read_excel(local_path, sheet_name='Employees'),
        'Products': pl.read_excel(local_path, sheet_name='Products'),
        'Orders': pl.read_excel(local_path, sheet_name='Orders')
    }
    
    print("\nSheet Contents:")
    print("=" * 80)
    
    for name, df in sheets.items():
        print(f"\n{name}:")
        print(df)
        print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")

## 6. Data Validation

In [ ]:
if local_path and os.path.exists(local_path):
    print("\nValidation Results:")
    print("=" * 80)
    
    validations = [
        ("Sheet count", len(sheets) == 3, f"Expected 3 sheets, got {len(sheets)}"),
        ("Employees records", sheets['Employees'].shape[0] == 3, f"Expected 3 rows, got {sheets['Employees'].shape[0]}"),
        ("Products records", sheets['Products'].shape[0] == 3, f"Expected 3 rows, got {sheets['Products'].shape[0]}"),
        ("Orders records", sheets['Orders'].shape[0] == 3, f"Expected 3 rows, got {sheets['Orders'].shape[0]}"),
        ("Employees columns", sheets['Employees'].shape[1] == 3, f"Expected 3 columns, got {sheets['Employees'].shape[1]}"),
        ("Products columns", sheets['Products'].shape[1] == 3, f"Expected 3 columns, got {sheets['Products'].shape[1]}"),
        ("Orders columns", sheets['Orders'].shape[1] == 3, f"Expected 3 columns, got {sheets['Orders'].shape[1]}")
    ]
    
    passed = 0
    failed = 0
    
    for test_name, result, message in validations:
        if result:
            print(f"✓ {test_name}: PASS")
            passed += 1
        else:
            print(f"✗ {test_name}: FAIL - {message}")
            failed += 1
    
    print("\n" + "=" * 80)
    print(f"Results: {passed} passed, {failed} failed")
    
    if failed == 0:
        print("\n🎉 All validations passed!")
    else:
        print("\n⚠️  Some validations failed. Check the results above.")

## 7. Sample Data Preview

In [ ]:
if local_path and os.path.exists(local_path):
    print("\nExpected Data Samples:")
    print("=" * 80)
    
    print("\nEmployees:")
    print("  1, Alice Johnson, Engineering")
    print("  2, Bob Smith, Sales")
    print("  3, Carol Williams, Marketing")
    
    print("\nProducts:")
    print("  101, Laptop, 999.99")
    print("  102, Mouse, 29.99")
    print("  103, Keyboard, 79.99")
    
    print("\nOrders:")
    print("  1001, John Doe, 2025-01-15")
    print("  1002, Jane Smith, 2025-01-16")
    print("  1003, Mike Brown, 2025-01-17")